# **Functions**

In [1]:
# Função para obter os dados da API
def get_data(url):
    try:
        # Fazendo a requisição GET
        response = requests.get(url)
        response.raise_for_status()  # Levanta um HTTPError para status de resposta de erro
        # Convertendo a resposta em JSON
        data = response.json()
        # Carregando os dados em um DataFrame do pandas
        df = pd.DataFrame(data)
        return df
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return None

# **Libraries**

In [53]:
import requests
import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt

# **Loading Data**

## Products

In [151]:
# URL da API para obter todos os produtos
url_products = 'https://fakestoreapi.com/products'

df_products = get_data(url_products)
df_products.head()

,id,title,price,description,category,image,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


In [152]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           20 non-null     int64  
 1   title        20 non-null     object 
 2   price        20 non-null     float64
 3   description  20 non-null     object 
 4   category     20 non-null     object 
 5   image        20 non-null     object 
 6   rating       20 non-null     object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ KB


In [157]:
df_products.to_csv('data/df_products.csv')
print('df_products salvo!')

df_products salvo!


## Carts

In [153]:
# URL da API para obter todos os carrinhos
url_carts = 'https://fakestoreapi.com/carts'

df_carts = get_data(url_carts)
df_carts.head()

,id,userId,date,products,__v
0,1,1,2020-03-02T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}, {'productId'...",0
1,2,1,2020-01-02T00:00:00.000Z,"[{'productId': 2, 'quantity': 4}, {'productId'...",0
2,3,2,2020-03-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 2}, {'productId'...",0
3,4,3,2020-01-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}]",0
4,5,3,2020-03-01T00:00:00.000Z,"[{'productId': 7, 'quantity': 1}, {'productId'...",0


In [154]:
df_carts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7 non-null      int64 
 1   userId    7 non-null      int64 
 2   date      7 non-null      object
 3   products  7 non-null      object
 4   __v       7 non-null      int64 
dtypes: int64(3), object(2)
memory usage: 412.0+ bytes


In [158]:
df_carts.to_csv('data/df_carts.csv')
print('df_carts salvo!')

df_carts salvo!


## Users

In [155]:
url_users = 'https://fakestoreapi.com/users'
df_users = get_data(url_users)
df_users.head()

,address,id,email,username,password,name,phone,__v
0,"{'geolocation': {'lat': '-37.3159', 'long': '8...",1,john@gmail.com,johnd,m38rmF$,"{'firstname': 'john', 'lastname': 'doe'}",1-570-236-7033,0
1,"{'geolocation': {'lat': '-37.3159', 'long': '8...",2,morrison@gmail.com,mor_2314,83r5^_,"{'firstname': 'david', 'lastname': 'morrison'}",1-570-236-7033,0
2,"{'geolocation': {'lat': '40.3467', 'long': '-3...",3,kevin@gmail.com,kevinryan,kev02937@,"{'firstname': 'kevin', 'lastname': 'ryan'}",1-567-094-1345,0
3,"{'geolocation': {'lat': '50.3467', 'long': '-2...",4,don@gmail.com,donero,ewedon,"{'firstname': 'don', 'lastname': 'romer'}",1-765-789-6734,0
4,"{'geolocation': {'lat': '40.3467', 'long': '-4...",5,derek@gmail.com,derek,jklg*_56,"{'firstname': 'derek', 'lastname': 'powell'}",1-956-001-1945,0


In [156]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   address   10 non-null     object
 1   id        10 non-null     int64 
 2   email     10 non-null     object
 3   username  10 non-null     object
 4   password  10 non-null     object
 5   name      10 non-null     object
 6   phone     10 non-null     object
 7   __v       10 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 772.0+ bytes


In [159]:
df_users.to_csv('data/df_users.csv')
print('df_users salvo!')

df_users salvo!


# **Data Cleaning**

- Check column types (esp. datetime)
- Check for nulls and duplicates
- Split columns if necessary

In [12]:
# Load saved data
df_products = pd.read_csv('data/df_products.csv', index_col=0)
df_carts = pd.read_csv('data/df_carts.csv', index_col=0)
df_users = pd.read_csv('data/df_users.csv', index_col=0)

## Data Cleaning - Products

In [13]:
df_products.head()

,id,title,price,description,category,image,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


### Checking column types

In [14]:
df_products.dtypes

id               int64
title           object
price          float64
description     object
category        object
image           object
rating          object
dtype: object

### Checking sizes

In [15]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           20 non-null     int64  
 1   title        20 non-null     object 
 2   price        20 non-null     float64
 3   description  20 non-null     object 
 4   category     20 non-null     object 
 5   image        20 non-null     object 
 6   rating       20 non-null     object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ KB


### Checking null values and duplicated rows

In [16]:
df_products.isnull().sum()

id             0
title          0
price          0
description    0
category       0
image          0
rating         0
dtype: int64

In [17]:
print(f"O número de linhas duplicadas é: {df_products.duplicated().sum()}")

O número de linhas duplicadas é: 0


### Split columns

- split 'rating' column into 2 new columns: avg_rating and count_ratings
- delete old 'rating' column
- its easier to perform calculations with those 2 metrics separated.

In [18]:
# Garantindo que a coluna 'rating' seja convertida para dicionário p/poder aplicar lambda
df_products['rating'] = df_products['rating'].apply(eval)

# Extraindo valores da coluna 'rating' e criando novas colunas
df_products['avg_rating'] = df_products['rating'].apply(lambda x: x['rate'])
df_products['count_ratings'] = df_products['rating'].apply(lambda x: x['count'])

# Removendo a coluna antiga 'rating'
df_products = df_products.drop(columns=['rating'])

# Exibindo as primeiras linhas do DataFrame para verificar
df_products.head()

,id,title,price,description,category,image,avg_rating,count_ratings
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,2.1,430
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,4.6,400


### Saving cleaned dataset

In [19]:
df_products.to_csv('data/df_products_cleaned.csv', index=False)

## Data Cleaning - Carts

In [20]:
df_carts.head()

,id,userId,date,products,__v
0,1,1,2020-03-02T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}, {'productId'...",0
1,2,1,2020-01-02T00:00:00.000Z,"[{'productId': 2, 'quantity': 4}, {'productId'...",0
2,3,2,2020-03-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 2}, {'productId'...",0
3,4,3,2020-01-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}]",0
4,5,3,2020-03-01T00:00:00.000Z,"[{'productId': 7, 'quantity': 1}, {'productId'...",0


### Checking types

In [21]:
df_carts.dtypes

id           int64
userId       int64
date        object
products    object
__v          int64
dtype: object

In [22]:
# Alterando coluna 'date' para formato 'datetime'
df_carts['date'] = pd.to_datetime(df_carts['date']).dt.strftime('%Y-%m-%d')

### Checking null values and duplicated rows

In [23]:
# Contando valores nulos por coluna
df_carts.isnull().sum()

id          0
userId      0
date        0
products    0
__v         0
dtype: int64

In [24]:
print(f"O número de linhas duplicadas é: {df_carts.duplicated().sum()}")

O número de linhas duplicadas é: 0


### Columns transformations

In [25]:
# Verificando se a coluna '__v' tem algum valor diferente
df_carts['__v'].unique()

array([0])

In [26]:
# Como ela não tem, será descartada
df_carts.drop(columns='__v', inplace=True)
df_carts.columns

Index(['id', 'userId', 'date', 'products'], dtype='object')

In [27]:
df1 = df_carts.copy()

# Garantindo que a coluna 'products' seja convertida para listas de dicionários
df1['products'] = df1['products'].apply(ast.literal_eval)

# Criando colunas para cada produto de 1 a 20
for product_id in range(1, 21):
    df1[f'product_{product_id}'] = df1['products'].apply(
        lambda products_list: sum(product['quantity'] for product in products_list if product['productId'] == product_id)
    )

# Removendo a coluna original 'products'
df1 = df1.drop(columns=['products'])

# Exibindo as primeiras linhas do DataFrame para verificar
df1.head()

,id,userId,date,product_1,product_2,product_3,product_4,product_5,product_6,product_7,...,product_11,product_12,product_13,product_14,product_15,product_16,product_17,product_18,product_19,product_20
0,1,1,2020-03-02,4,1,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,2020-01-02,10,4,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2,2020-03-01,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,3,2020-01-01,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,3,2020-03-01,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### Saving cleaned dataset

In [28]:
df1.to_csv('data/df_carts_cleaned.csv', index=False)

## Data Cleaning - Users

In [29]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   address   10 non-null     object
 1   id        10 non-null     int64 
 2   email     10 non-null     object
 3   username  10 non-null     object
 4   password  10 non-null     object
 5   name      10 non-null     object
 6   phone     10 non-null     object
 7   __v       10 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 720.0+ bytes


In [30]:
df_users.head()

,address,id,email,username,password,name,phone,__v
0,"{'geolocation': {'lat': '-37.3159', 'long': '8...",1,john@gmail.com,johnd,m38rmF$,"{'firstname': 'john', 'lastname': 'doe'}",1-570-236-7033,0
1,"{'geolocation': {'lat': '-37.3159', 'long': '8...",2,morrison@gmail.com,mor_2314,83r5^_,"{'firstname': 'david', 'lastname': 'morrison'}",1-570-236-7033,0
2,"{'geolocation': {'lat': '40.3467', 'long': '-3...",3,kevin@gmail.com,kevinryan,kev02937@,"{'firstname': 'kevin', 'lastname': 'ryan'}",1-567-094-1345,0
3,"{'geolocation': {'lat': '50.3467', 'long': '-2...",4,don@gmail.com,donero,ewedon,"{'firstname': 'don', 'lastname': 'romer'}",1-765-789-6734,0
4,"{'geolocation': {'lat': '40.3467', 'long': '-4...",5,derek@gmail.com,derek,jklg*_56,"{'firstname': 'derek', 'lastname': 'powell'}",1-956-001-1945,0


### Checking types

In [31]:
df_users.dtypes

address     object
id           int64
email       object
username    object
password    object
name        object
phone       object
__v          int64
dtype: object

### Checking null values and duplicated rows

In [32]:
# Contando valores nulos por coluna
df_users.isnull().sum()

address     0
id          0
email       0
username    0
password    0
name        0
phone       0
__v         0
dtype: int64

In [33]:
print(f"O número de linhas duplicadas é: {df_users.duplicated().sum()}")

O número de linhas duplicadas é: 0


### Column transformations

- Split 'address' column to extract 'lat' and 'long' data, as well other geolocation data
- Transform column 'name' to show only the complete name
- Delete '__v' column

In [34]:
df1 = df_users.copy()

# Convertendo a coluna 'address' de string para dicionário
df1['address'] = df1['address'].apply(ast.literal_eval)

# Extraindo e criando as novas colunas
df1['lat'] = df1['address'].apply(lambda x: float(x['geolocation']['lat']))
df1['long'] = df1['address'].apply(lambda x: float(x['geolocation']['long']))
df1['city'] = df1['address'].apply(lambda x: x['city'])
df1['street'] = df1['address'].apply(lambda x: x['street'])
df1['number'] = df1['address'].apply(lambda x: int(x['number']))
df1['zipcode'] = df1['address'].apply(lambda x: x['zipcode'])

# Removendo a coluna original 'address'
df1 = df1.drop(columns=['address'])

# Ajustando as colunas 'city' e 'street' para ter a primeira letra de cada palavra em maiúscula
df1['city'] = df1['city'].str.title()
df1['street'] = df1['street'].str.title()

# Exibindo as primeiras linhas do DataFrame para verificar
df1.head()

,id,email,username,password,name,phone,__v,lat,long,city,street,number,zipcode
0,1,john@gmail.com,johnd,m38rmF$,"{'firstname': 'john', 'lastname': 'doe'}",1-570-236-7033,0,-37.3159,81.1496,Kilcoole,New Road,7682,12926-3874
1,2,morrison@gmail.com,mor_2314,83r5^_,"{'firstname': 'david', 'lastname': 'morrison'}",1-570-236-7033,0,-37.3159,81.1496,Kilcoole,Lovers Ln,7267,12926-3874
2,3,kevin@gmail.com,kevinryan,kev02937@,"{'firstname': 'kevin', 'lastname': 'ryan'}",1-567-094-1345,0,40.3467,-30.1310,Cullman,Frances Ct,86,29567-1452
3,4,don@gmail.com,donero,ewedon,"{'firstname': 'don', 'lastname': 'romer'}",1-765-789-6734,0,50.3467,-20.1310,San Antonio,Hunters Creek Dr,6454,98234-1734
4,5,derek@gmail.com,derek,jklg*_56,"{'firstname': 'derek', 'lastname': 'powell'}",1-956-001-1945,0,40.3467,-40.1310,San Antonio,Adams St,245,80796-1234


In [35]:
# Deixando apenas o nome completo na coluna 'name'
df1['name'] = df1['name'].apply(lambda x: f"{eval(x)['firstname'].title()} {eval(x)['lastname'].title()}")
df1.head()

,id,email,username,password,name,phone,__v,lat,long,city,street,number,zipcode
0,1,john@gmail.com,johnd,m38rmF$,John Doe,1-570-236-7033,0,-37.3159,81.1496,Kilcoole,New Road,7682,12926-3874
1,2,morrison@gmail.com,mor_2314,83r5^_,David Morrison,1-570-236-7033,0,-37.3159,81.1496,Kilcoole,Lovers Ln,7267,12926-3874
2,3,kevin@gmail.com,kevinryan,kev02937@,Kevin Ryan,1-567-094-1345,0,40.3467,-30.1310,Cullman,Frances Ct,86,29567-1452
3,4,don@gmail.com,donero,ewedon,Don Romer,1-765-789-6734,0,50.3467,-20.1310,San Antonio,Hunters Creek Dr,6454,98234-1734
4,5,derek@gmail.com,derek,jklg*_56,Derek Powell,1-956-001-1945,0,40.3467,-40.1310,San Antonio,Adams St,245,80796-1234


In [36]:
# Removendo a coluna '__v'
df1.drop(columns='__v', inplace=True)

# Mudando a ordem das colunas no dataset
df1 = df1[['id', 'name', 'phone', 'lat', 'long', 'city', 'street', 'number', 'zipcode', 'email', 'username', 'password']]

In [37]:
df1.head()

,id,name,phone,lat,long,city,street,number,zipcode,email,username,password
0,1,John Doe,1-570-236-7033,-37.3159,81.1496,Kilcoole,New Road,7682,12926-3874,john@gmail.com,johnd,m38rmF$
1,2,David Morrison,1-570-236-7033,-37.3159,81.1496,Kilcoole,Lovers Ln,7267,12926-3874,morrison@gmail.com,mor_2314,83r5^_
2,3,Kevin Ryan,1-567-094-1345,40.3467,-30.1310,Cullman,Frances Ct,86,29567-1452,kevin@gmail.com,kevinryan,kev02937@
3,4,Don Romer,1-765-789-6734,50.3467,-20.1310,San Antonio,Hunters Creek Dr,6454,98234-1734,don@gmail.com,donero,ewedon
4,5,Derek Powell,1-956-001-1945,40.3467,-40.1310,San Antonio,Adams St,245,80796-1234,derek@gmail.com,derek,jklg*_56


### Saving cleaned dataset

In [38]:
df1.to_csv('data/df_users_cleaned.csv', index=False)

# **Exploratory Data Analysis** (EDA)

*Realizar uma análise exploratória dos dados para descobrir padrões, tendências, e informações úteis. Isso deve incluir o uso de estatísticas descritivas e a criação de visualizações como gráficos de barras, linhas, e possivelmente mapas, se os dados incluírem informações geográficas.*

## Load cleaned data

In [42]:
df_products = pd.read_csv('data/df_products_cleaned.csv')
df_carts = pd.read_csv('data/df_carts_cleaned.csv')
df_users = pd.read_csv('data/df_users_cleaned.csv')

## Descritive Statistics

In [52]:
# Estatística Descritiva das variáveis numéricas
print('Estatísticas Descritivas de Products: ')
print(df_products[['price', 'avg_rating', 'count_ratings']].describe().round(2))

print('\nEstatísticas Descritivas de Carts:')
print(df_carts.describe().round(2))

print('\nEstatísticas Descritivas de Users:')
print(df_users.describe().round(2))

Estatísticas Descritivas de Products: 
        price  avg_rating  count_ratings
count   20.00       20.00          20.00
mean   162.05        3.56         286.80
std    272.22        0.96         162.75
min      7.95        1.90          70.00
25%     15.24        2.90         143.75
50%     56.49        3.70         254.50
75%    110.96        4.53         400.00
max    999.99        4.80         679.00

Estatísticas Descritivas de Carts:
         id  userId  product_1  product_2  product_3  product_4  product_5  \
count  7.00    7.00       7.00       7.00       7.00        7.0       7.00   
mean   4.00    3.14       2.86       0.71       0.86        0.0       0.29   
std    2.16    2.41       3.63       1.50       2.27        0.0       0.76   
min    1.00    1.00       0.00       0.00       0.00        0.0       0.00   
25%    2.50    1.50       0.00       0.00       0.00        0.0       0.00   
50%    4.00    3.00       2.00       0.00       0.00        0.0       0.00   
75%    5.5

## Products Analysis

- Price distribution
- What is the product with highest price? And lowest?
- What is the median price? And the median price per category?
- Ratings distribution
- What is the production with highest ratings? And lowest?
- What is the median rating of all products? And per category?
- What is the producs with most ratings? And with lowest ratings?

In [54]:
df_products.columns

Index(['id', 'title', 'price', 'description', 'category', 'image',
       'avg_rating', 'count_ratings'],
      dtype='object')

## Cart Analysis

- Orders in a period = how many carts in a time period
- Total sold by product
- Total sold by product through time
- Number of products per cart
- Average ticket per cart
- Correlation between number of products per cart and avg ticket
- Most bought product

## Users Analysis

- Geographic distribution = where are they located?
- Number of cities with most users (ranking or map with proportional bubbles)
- What are cities with most active users = frequent buyers

# **References**

- **FakeStore API:** https://fakestoreapi.com/
- **Exporting notebooks as PDF files:** [MLJar](https://mljar.com/blog/jupyter-notebook-pdf/), [nbconvert docs](https://nbconvert.readthedocs.io/en/stable/)